In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('../')
import tokamakTK

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
import statsmodels.api as sm
import matplotlib.patches as mpatches
import matplotlib.colors as mcolors
import seaborn as sns
import plotly.express as px
import plotly.subplots as plsp
import plotly.graph_objects as go

from sklearn.decomposition import PCA
from plotly.subplots import make_subplots
from tokamakTK import MyCounter, HUEOrder
from collections import Counter
from statsmodels.stats.outliers_influence import variance_inflation_factor


pd.set_option('display.max_columns', None)
plt.rc('font',family = 'serif')
TD_colors = plt.cm.get_cmap('flag', 20) #Paired, flag
colors_ = sns.color_palette('viridis', 20)
CSS_colors = [mcolors.to_hex(TD_colors(i)) for i in range(20)]
path = "../data/"

/tmp/ipykernel_5780/4012704369.py:26: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  TD_colors = plt.cm.get_cmap('flag', 20) #Paired, flag


In [4]:
# Obtained from Optimization
min_subset_ids = pd.read_csv(path+"R_ids_alpha_0.6357.csv")

DB2 = pd.read_csv(path+"DB2P8.csv")
DB5 = pd.read_csv(path+"SELDB5_SVD.csv", low_memory=False) 

# Setting ELMy Dataset
DB5 = DB5[DB5["PHASE"].isin(['HGELM', 'HSELM', 'HGELMH', 'HSELMH'])]

# REMOVING SPHERICAL TOKAMAKS
#DB5 = DB5[~DB5.TOK.isin(['MAST', 'NSTX', 'START'])]

# There is two shots from DB2P8 missing in DB5
missing_shots = DB2[~DB2.id.isin( DB5.id.values )].reset_index(drop=True)
DB5 = pd.concat([DB5, missing_shots], axis=0, ignore_index=True)

# Labeling shots that had great impact in decreasing alpha_R
DB5.insert(loc=2,column="label",value=[0]*len(DB5))
DB5.insert(loc=2,column="label_str",value=["Unaffected"]*len(DB5))
DB5.loc[(DB5[DB5.id.isin(min_subset_ids.id)].index), "label"] = 1
DB5.loc[(DB5[DB5.id.isin(min_subset_ids.id)].index), "label_str"] = "Decreasing"

print(
    f"{ round( (len(min_subset_ids)/len(DB5))*100     ,2)  }% of the data decreased alpha_R\n" + 
    f"{ round( (1 - len(min_subset_ids)/len(DB5))*100 ,2)  }% of the data did not decrease alpha_R"
)

23.45% of the data decreased alpha_R
76.55% of the data did not decrease alpha_R


In [5]:
features = ['IP', 'BT', 'NEL', 'PLTH', 'RGEO', 'KAREA', 'EPS', 'MEFF']

In [6]:
print(DB5[features].head(5))

       IP     BT    NEL   PLTH   RGEO   KAREA       EPS  MEFF
0  0.2959  2.205  3.789  1.870  1.694  0.9748  0.242090   1.5
1  0.2952  2.205  3.734  2.024  1.684  0.9724  0.244240   1.5
2  0.2971  2.205  3.410  1.132  1.693  0.9895  0.238807   1.5
3  0.2959  2.205  3.775  1.376  1.693  0.9744  0.242351   1.5
4  0.2942  2.204  3.847  2.033  1.685  0.9691  0.244926   1.5


In [5]:
tokamakTK.get_regression(DB5, DB2, withDB2=True)[1].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.941
Model:                            OLS   Adj. R-squared:                  0.941
Method:                 Least Squares   F-statistic:                 1.251e+04
Date:                Tue, 23 May 2023   Prob (F-statistic):               0.00
Time:                        16:31:13   Log-Likelihood:                 1142.2
No. Observations:                6252   AIC:                            -2266.
Df Residuals:                    6243   BIC:                            -2206.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.5185      0.029    -88.250      0.000      -2.574      -2.463
x1             1.1343      0.016     69.351      0.000       1.102       1.166
x2             0.0724      0.014      5.079      0.000       0.044       0.100
x3             0.1749      0.010     18.248      0.000       0.156       0.194
x4            -0.6827      0.007   -100.123      0.000      -0.696      -0.669
x5             1.4479      0.026     56.757      0.000       1.398       1.498
x6             0.2802      0.029      9.731      0.000       0.224       0.337
x7             0.0636      0.037      1.720      0.085      -0.009       0.136
x8             0.2140      0.017     12.541      0.000       0.181       0.247
==============================================================================
Omnibus:                      104.112   Durbin-Watson:                   0.571
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              170.550
Skew:                          -0.149   Prob(JB):                     9.24e-38
Kurtosis:                       3.753   Cond. No.                         64.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### OLS

```Python
X = DB5[features].apply(np.abs).apply(np.log)
X.insert(0,"intercept",np.ones(len(X))); X = X.to_numpy()
y = DB5[["TAUTH"]].apply(np.log).to_numpy()

B = np.matmul( np.linalg.inv( np.matmul(X.T,X) ) ,  np.matmul(X.T,y))
```

### Without Intercept

[Notes based on PYTHON FOR DATA SCIENCE](https://www.pythonfordatascience.org/linear-regression-python/)

### log( abs( X ) ) | With Spherical DS | complete DB5

In [6]:
X = DB5[features].apply(np.abs).apply(np.log).to_numpy()
tokamakTK.get_multicollinearity_info(X, features)

,eigenvalues,CI,VIF,VD
IP,0.993224,1.000000,17.088996,0.637650
BT,0.240220,3.842289,16.498704,0.154221
NEL,0.173599,7.183002,41.686928,0.111451
PLTH,0.085816,8.639261,26.961506,0.055094
RGEO,0.002671,12.800543,62.618108,0.001715
KAREA,0.030393,19.202273,24.294116,0.019512
EPS,0.021643,28.966310,198.242717,0.013895
MEFF,0.010065,56.758544,16.823753,0.006462


### X | With Spherical DS | complete DB5

In [7]:
X = DB5[features].to_numpy()
tokamakTK.get_multicollinearity_info(X, features)

,eigenvalues,CI,VIF,VD
IP,30.565286,1.000000,3.412519,0.675419
BT,10.850580,2.907415,2.401006,0.239772
NEL,2.310020,7.030995,6.607237,0.051046
PLTH,1.296787,9.544358,6.450341,0.028656
RGEO,0.120138,10.855804,34.628852,0.002655
KAREA,0.076445,37.076629,90.215626,0.001689
EPS,0.003641,59.892198,27.303791,0.000080
MEFF,0.030912,190.269981,48.199571,0.000683


### X | With Spherical DS | Decreasing DB5 + DB2

In [8]:
X = DB5[ (DB5.label.isin([1])) |  (DB5.id.isin(DB2.id.values)) ][features].to_numpy()
tokamakTK.get_multicollinearity_info(X, features)

,eigenvalues,CI,VIF,VD
IP,21.516647,1.000000,2.631923,0.518633
BT,15.615864,2.514931,1.681189,0.376402
NEL,2.695337,4.766412,4.701631,0.064968
PLTH,1.438796,8.187015,7.067288,0.034680
RGEO,0.094818,10.013100,31.690852,0.002285
KAREA,0.077863,34.808049,54.091793,0.001877
EPS,0.004347,46.704140,21.126279,0.000105
MEFF,0.043563,145.070718,42.795358,0.001050


### X | With Spherical DS | Unaffected DB5 + DB2

In [9]:
X = DB5[ DB5.label.isin([0]) ][features].to_numpy()
tokamakTK.get_multicollinearity_info(X, features)

,eigenvalues,CI,VIF,VD
IP,34.176351,1.000000,3.234462,0.687963
BT,11.502419,2.846503,2.235422,0.231541
NEL,2.461138,6.757870,6.482577,0.049542
PLTH,1.304755,9.052530,5.999891,0.026264
RGEO,0.126331,11.308441,33.311640,0.002543
KAREA,0.069598,38.234495,77.582911,0.001401
EPS,0.003654,62.161721,26.462298,0.000074
MEFF,0.033348,190.792680,53.209751,0.000671
